In [1]:
import tensorflow as tf
import keras
import keras.backend as kb
from keras.models import Sequential, Model
from keras.layers import (
    Activation, 
    BatchNormalization, 
    Concatenate,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    Input,
    MaxPooling2D
)
from keras.optimizers import TFOptimizer, Adam, SGD
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
def create_mlp(dim):
    # define our MLP network
    model = Sequential()
    model.add(Dense(64, input_dim=dim, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(128, input_dim=dim, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(64, input_dim=dim, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    # return our model
    return model

def create_cnn(height, width, depth, filters=(16, 32, 64), regress=False):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)

    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs
            filt = (5,5)
        else:
            filt = (3,3)
        # CONV => RELU => BN => POOL
        
        x = Conv2D(f, filt, padding="same")(x)
        x = Activation("relu")(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = BatchNormalization()(x)
    
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    
    
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(128)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(64)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model

def CRPS(yTrue, yPred):
    yPred = kb.cumsum(yPred, axis=1)
    return kb.mean(kb.sum(kb.square(yPred - yTrue), axis=1)) / 199

In [3]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, imageDim=(100,200),
                 nChannels = 4, gsDim = 37, shuffle=True):
        'Initialization'
        self.imageDim = imageDim
        self.gsDim = gsDim
        self.nChannels = nChannels
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        Xcombined, yardage = self.__data_generation(list_IDs_temp)

        return Xcombined, yardage

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        Xim = np.empty((self.batch_size, *self.imageDim, self.nChannels))
        Xgs = np.empty((self.batch_size, self.gsDim))
        yardage = np.empty((self.batch_size,199), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            temp = np.load('data/files/image' + ID + '.npy')
            Xim[i,] = temp.astype('float')/128.
            Xgs[i,] = np.load('data/files/gameState' + ID + '.npy')
            
            # Store output
            yardage[i,] = np.load('data/files/yardage' + ID + '.npy')

        return [Xgs,Xim], yardage

In [4]:
partition = np.load('data/partition_dict.npy',allow_pickle = 'TRUE').item()
batch_size = 64
gsDim = 37
training_generator = DataGenerator(
    partition['train'], 
    batch_size = batch_size)
validation_generator = DataGenerator(
    partition['validation'], 
    batch_size = batch_size)
testing_generator = DataGenerator(
    partition['test'], 
    batch_size = batch_size)

In [5]:
mlp = create_mlp(gsDim)
cnn = create_cnn(100, 200, 4)

x = Concatenate()([mlp.output, cnn.output])

x = BatchNormalization()(x)
x = Dense(128, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)

x = Dense(199, activation="softmax")(x)

model = Model(inputs=[mlp.input, cnn.input], 
              outputs=x)
model.compile(loss=CRPS,
              metrics = [CRPS],
              optimizer=Adam())

In [7]:
combined_json = model.to_json()
with open("combined.json", "w") as json_file:
    json_file.write(combined_json)

In [80]:
checkpointer = keras.callbacks.ModelCheckpoint(filepath='./combined/combinedWeights.hdf5', verbose=1, save_best_only=False)

es = EarlyStopping(monitor='val_loss', 
                   mode='min',
                   restore_best_weights=True, 
                   verbose=1, 
                   patience=11)
es.set_model(model)

lr = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.5,
                       patience=5,
                       verbose=1,
                       mode='min',
                       min_delta=0.00001)

history = model.fit_generator(generator = training_generator,
                   validation_data = validation_generator,
                   epochs = 2,
                   callbacks = [es, lr, checkpointer],
                   verbose = 1)

Epoch 1/2
760/760 [==============================] - 1285s 2s/step - loss: 0.0215 - CRPS: 0.0215 - val_loss: 0.0181 - val_CRPS: 0.0148

Epoch 00001: saving model to ./combined/combinedWeights.hdf5
Epoch 2/2
760/760 [==============================] - 1248s 2s/step - loss: 0.0153 - CRPS: 0.0153 - val_loss: 0.0142 - val_CRPS: 0.0144

Epoch 00002: saving model to ./combined/combinedWeights.hdf5


In [72]:
score = model.evaluate_generator(testing_generator, verbose = 1)

162/162 [==============================] - 54s 334ms/step


In [73]:
np.save('./combined/comb_test_score.npy',score)
np.save('./combined/comb_hist.npy',history)

[0.015449351631104946, 0.01481718197464943]